In [5]:
from selenium import webdriver
from urllib import request
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import re
import sys
import json

In [6]:
sys.dont_write_bytecode = True
sys.path.append('../')
from tools.maneger import DataManager

In [7]:
corpus_root = "../../corpus"
name = "question"
data_path = "/".join([corpus_root, name]) + "/"

corpus_name = "What.json"
print(data_path)

../../corpus/question/


In [8]:
dataM = DataManager(data_path)

In [9]:
class Crawler:
    def __init__(self, ) -> None:
        self.YN_stop_list = "Is Do Did Can Have".split()
        self.WH_stop_list = "What When Where Who What Why How".split()
        self.please_stop_list = ["please", "Would you", "Could you"]
        self.plain_stop_list = "I She He They We You".split()
    
    def connect(self, server=None):
        chrome_options = webdriver.ChromeOptions()
        if server:
            command = "http://{0}:4444/wd/hub".format(server)
            self.driver = webdriver.Remote(
                command_executor="http://{0}:4444/wd/hub".format(server),
                options=chrome_options
            )
    
    def crawl(self, mode, num):
        self.mode = mode
        self.num = num
        if mode == "YN":
            self.stop_list = self.YN_stop_list
            self.essential = "?"
            self._crawl_ENG()
        elif mode == "WH":
            self.stop_list = self.WH_stop_list
            self.essential = "?"
            self._crawl_ENG()
        elif mode =="please":
            self.stop_list = self.please_stop_list
            self.essential = "?"
            self._crawl_ENG()
        elif mode == "plain":
            self.stop_list = self.plain_stop_list
            self.essential = "."
            self._crawl_ENG()
        
    
    def _in(self, search, target):
        """
        search : list or str
        target : str
        """
        result = True
        if isinstance(search, list):
            for s in search:
                result = result and s in target
            return result
        elif isinstance(search, str):
            return search in target
        else:
            print("list or str is expected, but not")
            return False
    
    def _extract_stop_type(self, target):
        if self.essential not in target:
            return None
        for stop in self.stop_list:
            if stop in target:
                return stop
        return None
    
    def _extract_label(self, text):
        if self.mode == "YN":
            return ["YN", self._extract_stop_type(text)]
        # 後で
        elif self.mode == "WH":
            return ["WH", self._extract_stop_type(text)]
        elif self.mode == "plain":
            return ["plain"]
        else:
            return ["please"]
    
    def _go_conv_page(self):
        element = self.driver.find_element_by_class_name("condition").find_elements_by_tag_name("a")
        element[1].click()
        
    def _crawl_ENG(self):
        self.data = []
        self.label = []

        self.driver.get("https://gogakuru.com/english/phrase/genre/index.html")
        
        self._go_conv_page()

        html = self.driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        while 1:
            cells = soup.find_all("td", class_="summary")
            for cell in cells:
                engs = cell.find_all("span", class_="font-en")
                text_ = [eng.get_text() for eng in engs]
                text = text_[0]
                # print(text)
                # 大文字で含まれるならば
                stop = self._extract_stop_type(text)
                if  stop:
                    jp = cell.find("dd", class_="jp").get_text()
                    self.data.append(jp)
                    self.label.append( self._extract_label(text) )
                    print("stop:{0}, jp:{1}".format(stop, jp))

            # 一度打ち切り
            if len(self.data) >= self.num:
                break
        
            # 次のurl
            try:
                next_t = self.driver.find_element_by_class_name("right").find_element_by_tag_name('a')
                next_t.click()
            except:
                break
            html = self.driver.page_source.encode('utf-8')
            soup = BeautifulSoup(html, 'lxml')
        
        

    def make_corpus(self):
        self.corpus = {}
        self.corpus[self.mode] = []
        for t, l in zip(self.data, self.label):
            one_data = {"data":t, "label":l}
            self.corpus[self.mode].append(one_data)
    
    def write(self, filename):
        with open(filename, "w") as f:
            json.dump(self.corpus, f,  ensure_ascii=False, indent=4)
    
    def quit(self):
        self.driver.quit() 



In [10]:
corpus_root = "../../corpus"
# name = "question/short"
name = "question"
data_path = "/".join([corpus_root, name]) + "/"
data_path
dataM = DataManager(data_path)

In [1]:
crawler = Crawler()
mode = "plain"

corpus_name = "{0}.json".format(mode)

crawler.connect("192.168.1.224")

NameError: name 'Crawler' is not defined

In [20]:
crawler.crawl(mode,900)

stop:please, jp:メニューを見せていただけますか？
stop:please, jp:おいでいただけますか？
stop:please, jp:マッシュポテトをもらえますか？
stop:please, jp:伝言を預かっていただけますか？
stop:Would you, jp:ご一緒しませんか？
stop:Would you, jp:お代わりする？
stop:Would you, jp:お手伝いいただくことはできますか？
stop:Would you, jp:もう少しトーストはどう、ルナ？
stop:Would you, jp:サンドウイッチはいかがですか？
stop:Could you, jp:もう少し詳しく説明していただけますか。
stop:Could you, jp:もう一度言っていただけますか。
stop:Would you, jp:いっしょに来ますか？
stop:Would you, jp:もう一度チャンスをくれないかな？
stop:Would you, jp:私がご一緒してもいいですか。
stop:Would you, jp:いくつかお試しになりたいですか。
stop:Could you, jp:どれかすすめていただけますか。
stop:Could you, jp:「遅れそうです。席をとっておいてくれますか？舞台が始まる前には着くはずです」
stop:Would you, jp:今夜コンサートに行きたいですか？
stop:Could you, jp:もう一度あの引用文を言ってくれる？
stop:Could you, jp:少し小さな声で話してくれる？
stop:Could you, jp:クローディアが自分の財布を捜して遺失物取扱所に電話したとき、顧客係は「それはどんなものだったのか私に説明してもらえますか？」と尋ねました。
stop:Could you, jp:彼女への電話はあとにしてもらえますか？
stop:Would you, jp:あなたが先に行きますか？
stop:Would you, jp:「セス、前にいるこの男性で私はステージが見えません。私と席を変わってくれますか？」
stop:Would you, jp:「今夜、私たちと一緒に飲みに行きませんか？」
「お誘いありがとう。でも遠慮します」
stop:Would you, jp:お

In [21]:
crawler.make_corpus()
crawler.write(data_path+corpus_name)


In [22]:
crawler.quit()